「PyTorch入門  6. 最適化」
===============================================================
【原題】OPTIMIZING MODEL PARAMETERS

【原著】
[Suraj Subramanian](https://github.com/suraj813)、[Seth Juarez](https://github.com/sethjuarez/) 、[Cassie Breviu](https://github.com/cassieview/) 、[Dmitry Soshnikov](https://soshnikov.com/)、[Ari Bornstein](https://github.com/aribornstein/)


【元URL】https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2021年03月20日

【チュトーリアル概要】

本チュートリアルでは、オプティマイザー（Optimizer）を使用した、パラメータの最適化（≒学習）について解説を行います。

---


パラメータの最適化
===========================

モデルとデータを用意できたので続いてはモデルを訓練、検証することで、データに対してモデルのパラメータを最適化し、テストを行います。




モデルの訓練は反復的なプロセスとなります。

各イテレーション（エポックと呼ばれます）で、モデルは出力を計算し、損失を求めます。そして各パラメータについて損失に対する偏微分の値を求めます。

その後、勾配降下法に基づいてパラメータを最適化します。

この最適化プロセスの流れについては、以下の動画も参考にご覧ください。

- 動画：[backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)

---


コード準備
-----------------
入門シリーズの「2. データセットとデータローダー」および、「4. モデル構築」からコードを再利用します。


In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:01<00:00, 19.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 295kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.59MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.33MB/s]


---



ハイパーパラメータ
-----------------

ハイパーパラメータは、モデルの最適化プロセスを制御するためのパラメータです。

ハイパーパラメータの値が異なると、モデルの学習や収束率に影響します（詳細なハイパーパラメータチューニングの解説は[こちら](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)をご覧ください）。




今回は、訓練用のハイパーパラメータとして以下の値を使用します。

 - **Number of Epochs**：イテレーション回数
 - **Batch Size**：ミニバッチサイズを構成するデータ数
 - **Learning Rate**：パラメータ更新の係数。値が小さいと変化が少なく、大きすぎると訓練に失敗する可能性が生まれる


In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

---


最適化ループ
-----------------
ハイパーパラメータを設定後、訓練で最適化のループを回すことで、モデルを最適化します。

最適化ループの1回のイテレーションは、**エポック(epoch)**と呼ばれます。

各エポックでは2種類のループから構成されます。

 - **訓練ループ**：データセットに対して訓練を実行し、パラメータを収束させます

 - **検証 / テストループ**：テストデータセットでモデルを評価し、性能が向上しているか確認します




訓練ループ内で使用される概念について、簡単に把握しておきましょう。

本チュートリアルの最後には、最適化ループの完全な実装を紹介します。

**損失関数：Loss Function**

データが与えられても、訓練されていないネットワークは正しい答えを出力しない可能性があります。

損失関数はモデルが推論した結果と、実際の正解との誤差の大きさを測定する関数です。訓練ではこの損失関数の値を小さくしていきます。

損失を計算するためには、入力データに対するモデルの推論結果を求め、その値と正解のラベルとの違いを比較します。



一般的な損失関数としては、回帰タスクでは[`nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)(Mean Square Error)、分類タスクでは[`nn.NLLLoss`](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)(Negative Log Likelihood) が使用されます。

[`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)は、``nn.LogSoftmax`` と ``nn.NLLLoss``を結合した損失関数となります。

モデルが出力する`logit`値を`nn.CrossEntropyLoss`に与えて正規化し、予測誤差を求めます。

In [4]:
# loss functionの初期化、定義
loss_fn = nn.CrossEntropyLoss()

**最適化器：Optimizer**

最適化は各訓練ステップにおいてモデルの誤差を小さくなるように、モデルパラメータを調整するプロセスです。

<br>

**最適化アルゴリズム：Optimization algorithms**

最適化アルゴリズムは、最適化プロセスの具体的な手続きです（本チュートリアルでは確率的勾配降下法：Stochastic Gradient Descentを使用します）。

最適化のロジックは全て``optimizer``オブジェクト内に隠ぺいされます。



 今回はSGD optimizerを使用します。ただし、最適化関数にはADAMやRMSPropなど、様々な種類があります。

 詳細については、[こちら](https://pytorch.org/docs/stable/optim.html)を参照ください。



訓練したいモデルパラメータをoptimizerに登録し、合わせて学習率をハイパーパラメータとして渡すことで初期化を行います。




In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

訓練ループ内で、最適化（optimization）は3つのステップから構成されます。

[1] ``optimizer.zero_grad()``を実行し、モデルパラメータの勾配をリセットします。

勾配の計算は蓄積されていくので、毎イテレーション、明示的にリセットします。

<br>

[2] 続いて、``loss.backwards()``を実行し、バックプロパゲーションを実行します。

PyTorchは損失に対する各パラメータの偏微分の値（勾配）を求めます。

<br>

[3] 最後に、``optimizer.step()``を実行し、各パラメータの勾配を使用してパラメータの値を調整します。



---


実装全体：Full Implementation
-----------------------
最適化を実行するコードをループする``train_loop``と、テストデータに対してモデルの性能を評価する``test_loop``を定義します。




In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

損失関数とoptimizerを初期化し、それを ``train_loop`` と ``test_loop`` に渡します。

<br>

以下の実装例において、モデルの性能を向上させるために、`epoch`数は自由に変えてみてください。





In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305953  [    0/60000]
loss: 2.294588  [ 6400/60000]
loss: 2.288833  [12800/60000]
loss: 2.289989  [19200/60000]
loss: 2.281523  [25600/60000]
loss: 2.267299  [32000/60000]
loss: 2.281313  [38400/60000]
loss: 2.258862  [44800/60000]
loss: 2.245963  [51200/60000]
loss: 2.267616  [57600/60000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 0.035258 

Epoch 2
-------------------------------
loss: 2.234504  [    0/60000]
loss: 2.239228  [ 6400/60000]
loss: 2.221017  [12800/60000]
loss: 2.243783  [19200/60000]
loss: 2.224848  [25600/60000]
loss: 2.183587  [32000/60000]
loss: 2.228232  [38400/60000]
loss: 2.179879  [44800/60000]
loss: 2.152042  [51200/60000]
loss: 2.198169  [57600/60000]
Test Error: 
 Accuracy: 42.9%, Avg loss: 0.033830 

Epoch 3
-------------------------------
loss: 2.145478  [    0/60000]
loss: 2.155267  [ 6400/60000]
loss: 2.109789  [12800/60000]
loss: 2.147364  [19200/60000]
loss: 2.124945  [25600/60000]
loss: 2.059281  [32000/600

さらなる詳細
--------------
以下のページも参考ください。

- [`Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [`torch.optim](https://pytorch.org/docs/stable/optim.html)
- [`Warmstart Training a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)




以上。